In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [12]:
import glob, os
import yaml
import numpy as np
import pandas as pd

In [3]:
%cd /content/drive/MyDrive/c170m_final_project/Globus_data

/content/drive/MyDrive/c170m_final_project/Globus_data


In [4]:
# Find number of files in directory
globus_dir = "/content/drive/MyDrive/c170m_final_project/Globus_data"
list_plz = os.listdir(globus_dir)

In [5]:
# Make array with correct ordering of files
file_indices = np.array([0], dtype=np.int64)

for i in range(1000):
    if i%1000 == 0:
        file_indices = np.append(file_indices, int((1000+i)/1000))
        file_indices = np.append(file_indices, int((1000+i)/100))
        file_indices = np.append(file_indices, int((1000+i)/10))
        file_indices = np.append(file_indices, int(1000+i))
    elif i%100 == 0:
        file_indices = np.append(file_indices, int((1000+i)/100))
        file_indices = np.append(file_indices, int((1000+i)/10))
        file_indices = np.append(file_indices, int(1000+i))
    elif i%10 == 0:
        file_indices = np.append(file_indices, int((1000+i)/10))
        file_indices = np.append(file_indices, int(1000+i))
    else:
        file_indices = np.append(file_indices, int(1000+i))

for i in range(200, 1000):
    if i%100 == 0:
        file_indices = np.append(file_indices, int(i/100))
        file_indices = np.append(file_indices, int(i/10))
        file_indices = np.append(file_indices, int(i))
    elif i%10 == 0:
        file_indices = np.append(file_indices, int(i/10))
        file_indices = np.append(file_indices, int(i))
    else:
        file_indices = np.append(file_indices, int(i))

print(len(file_indices))

2000


In [6]:
# Function to extract void params from each folder
def get_void_params(nn_params, file_num):

    # Loop through files in given folder and read 'em
    for file in glob.glob("*.out"):
        #print(file)

        # Counter to distnguish between different files
        i = -1

        # Open and generate data from file
        data = np.genfromtxt(file, skip_header=0)
        #print(data)
        if file[0] == "s":
            i = 0
        elif file[0] == "c":
            i = 1
        else:
            print("Invalid file encountered, stopping run:", file)
            break

        # Extract params
        if i == 0:
            # Extract ellipticity from shapes data
            ellip = data[:,1]  # Ellipticity is in 2nd column of data
        if i == 1:
            # Extract radius and density contrast from centers data
            radius = data[:,4]  # Radius (Mpc/h) is in 5th column of data
            density_contrast = data[:,8]  # Density contrast is in 9th column of data

    # Stack params and concatenate to nn_params
    index_no = np.full(shape=len(ellip), fill_value=file_num, dtype=np.int64)
    add_params = np.c_[index_no, ellip, radius, density_contrast]
    nn_params = np.concatenate((nn_params, add_params))
    return nn_params

In [7]:
# Declare empty np array to store params in order: ellipticity, radius, density contrast
# Column headers: -1 -> Index, 0 -> ellipticity, 1 -> radius, 2 -> density contrast
nn_inputs = np.array([[-1, 0, 1, 2]])

In [8]:
#get_void_params(glob.glob("*.out"), nn_inputs)
current_dir = os.getcwd()
print(current_dir)

/content/drive/MyDrive/c170m_final_project/Globus_data


In [9]:
# Sample for loop to go in and out of folders
for file_num in file_indices:
    if file_num == 1897:  # For some reason, this folder doesn't have the files we need
                          # Fill each column with 605 NaNs
        index = np.full(shape=605, fill_value=file_num, dtype=np.int64)
        nans = np.full(shape=605, fill_value=np.nan)
        add_inputs = np.c_[index, nans, nans, nans]
        nn_inputs = np.concatenate((nn_inputs, add_inputs))
        continue

    # Enter folder
    %cd 'sample_Quijote_'{file_num}'_ss1.0_z0.00_d00'
    #%ls

    # Extract params
    nn_inputs = get_void_params(nn_inputs, file_num)

    # Exit folder
    %cd ..

/content/drive/MyDrive/c170m_final_project/Globus_data/sample_Quijote_0_ss1.0_z0.00_d00
/content/drive/MyDrive/c170m_final_project/Globus_data
/content/drive/MyDrive/c170m_final_project/Globus_data/sample_Quijote_1_ss1.0_z0.00_d00
/content/drive/MyDrive/c170m_final_project/Globus_data
/content/drive/MyDrive/c170m_final_project/Globus_data/sample_Quijote_10_ss1.0_z0.00_d00
/content/drive/MyDrive/c170m_final_project/Globus_data
/content/drive/MyDrive/c170m_final_project/Globus_data/sample_Quijote_100_ss1.0_z0.00_d00
/content/drive/MyDrive/c170m_final_project/Globus_data
/content/drive/MyDrive/c170m_final_project/Globus_data/sample_Quijote_1000_ss1.0_z0.00_d00
/content/drive/MyDrive/c170m_final_project/Globus_data
/content/drive/MyDrive/c170m_final_project/Globus_data/sample_Quijote_1001_ss1.0_z0.00_d00
/content/drive/MyDrive/c170m_final_project/Globus_data
/content/drive/MyDrive/c170m_final_project/Globus_data/sample_Quijote_1002_ss1.0_z0.00_d00
/content/drive/MyDrive/c170m_final_project

In [14]:
print(nn_inputs.shape)

(4284719, 4)


In [18]:
df = pd.DataFrame(nn_inputs)
df = df.drop(0)

headers = ["index", "ellipticity", "radius", "density_contrast"]
df.columns = headers

         index  ellipticity  radius  density_contrast
1          0.0     0.198568  129.17          3.487631
2          0.0     0.162464  172.36          3.154508
3          0.0     0.118820   83.54          2.724835
4          0.0     0.120590   75.79          2.692753
5          0.0     0.101951   64.01          2.678991
...        ...          ...     ...               ...
4284714  999.0     0.131346   18.75          1.000166
4284715  999.0     0.183111   17.07          1.000152
4284716  999.0     0.196843   28.28          1.000123
4284717  999.0     0.213002   16.22          1.000094
4284718  999.0     0.108824   25.75          1.000037

[4284718 rows x 4 columns]


In [25]:
df.to_csv("/content/drive/MyDrive/c170m_final_project/nn_input_params.csv")